In [1]:
!pip install h2o==3.34.0.8

     |████████████████████████████████| 175.8 MB 11 kB/s              
  Preparing metadata (setup.py) ... - done
  Created wheel for h2o: filename=h2o-3.34.0.8-py2.py3-none-any.whl size=175904398 sha256=3719ff08a06dc7fd38951ff57349506e36e6fa57190fb85ff88b720b8f2e95f2
  Stored in directory: /root/.cache/pip/wheels/f0/2e/33/4366d488c23b0c08ae68accd8b26612a3dc59c84bdb13a9383
Successfully built h2o
  Attempting uninstall: h2o
    Found existing installation: h2o 3.36.0.2
    Uninstalling h2o-3.36.0.2:
      Successfully uninstalled h2o-3.36.0.2


In [2]:
import pandas as pd
import os
import numpy as np
import h2o
from h2o.automl import H2OAutoML
SEED=6789

# Pre Processing

First we applied some post processing using formula : EUI = previousEUI - previous*(energy_star_rating - previous_energy_star_rating)/150

In [3]:
train=pd.read_csv("../input/widsdatathon2022/train.csv")
test=pd.read_csv("../input/widsdatathon2022/test.csv")
sub=pd.read_csv('../input/part-1-wids2022-model-with-lag/submission_automl1.csv')

df=train
cols1=['State_Factor','building_class','facility_type','floor_area','year_built']

In [4]:
df=df.sort_values(by=cols1+['Year_Factor'])
df['site_eui_lag1']=df.groupby(cols1)['site_eui'].shift(1)
df['Year_Factor_lag1']=df.groupby(cols1)['Year_Factor'].shift(1)
df['energy_star_rating_lag1']=df.groupby(cols1)['energy_star_rating'].shift(1)

df['delta_energy'] = (df['energy_star_rating']-df['energy_star_rating_lag1'] ) / df['energy_star_rating']
df['delta_energy2'] = (df['energy_star_rating']-df['energy_star_rating_lag1'] ) / 100
df['delta_site_eui'] = (df['site_eui']-df['site_eui_lag1'] ) /df['site_eui']
df['delta_Year_Factor'] = (df['Year_Factor']-df['Year_Factor_lag1'] ) 

toto1=df[cols1+['site_eui','Year_Factor','energy_star_rating']].groupby(cols1)['site_eui','Year_Factor','energy_star_rating'].last().reset_index().rename(columns={'site_eui':'site_eui_cols1',
                                                                                                                       'Year_Factor':'Year_Factor_cols1',
                                                                                                                        'energy_star_rating':'energy_star_rating_cols1'})

test=test.merge(toto1,on=cols1,how="left")

test.head()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  # This is added back by InteractiveShellApp.init_path()


,Year_Factor,State_Factor,building_class,facility_type,floor_area,year_built,energy_star_rating,ELEVATION,january_min_temp,january_avg_temp,...,days_above_100F,days_above_110F,direction_max_wind_speed,direction_peak_wind_speed,max_wind_speed,days_with_fog,id,site_eui_cols1,Year_Factor_cols1,energy_star_rating_cols1
0,7,State_1,Commercial,Grocery_store_or_food_market,28484.0,1994.0,37.0,2.4,38,50.596774,...,2,0,NaN,NaN,NaN,NaN,75757,168.039617,6.0,NaN
1,7,State_1,Commercial,Grocery_store_or_food_market,21906.0,1961.0,55.0,45.7,38,50.596774,...,2,0,NaN,NaN,NaN,NaN,75758,272.553279,6.0,48.0
2,7,State_1,Commercial,Grocery_store_or_food_market,16138.0,1950.0,1.0,59.1,38,50.596774,...,2,0,NaN,NaN,NaN,NaN,75759,151.983607,6.0,1.0
3,7,State_1,Commercial,Grocery_store_or_food_market,97422.0,1971.0,34.0,35.4,38,50.596774,...,2,0,NaN,NaN,NaN,NaN,75760,264.375683,6.0,32.0
4,7,State_1,Commercial,Grocery_store_or_food_market,61242.0,1942.0,35.0,1.8,38,50.596774,...,2,0,340.0,330.0,22.8,126.0,75761,242.535519,6.0,34.0


In [5]:
test['site_eui_sub']=sub['site_eui']


In [6]:
test['site_eui_cols1_updated'] = np.where((test['energy_star_rating'].notnull() ) & (test['energy_star_rating_cols1'].notnull() ) ,
                                           test['site_eui_cols1'] - test['site_eui_cols1'] * ( test['energy_star_rating'] - test['energy_star_rating_cols1'] ) / 150 ,
                                         test['site_eui_cols1'] )

In [7]:
test[['id','site_eui_sub','site_eui_cols1','site_eui_cols1_updated','Year_Factor_cols1','energy_star_rating','energy_star_rating_cols1']].head()

,id,site_eui_sub,site_eui_cols1,site_eui_cols1_updated,Year_Factor_cols1,energy_star_rating,energy_star_rating_cols1
0,75757,163.202741,168.039617,168.039617,6.0,37.0,NaN
1,75758,258.325591,272.553279,259.834126,6.0,55.0,48.0
2,75759,173.828553,151.983607,151.983607,6.0,1.0,1.0
3,75760,262.820162,264.375683,260.850674,6.0,34.0,32.0
4,75761,247.748933,242.535519,240.918616,6.0,35.0,34.0


In [8]:
test['Year_Factor_cols1'].value_counts()

6.0    8488
5.0     431
4.0      52
3.0      13
1.0       3
2.0       2
Name: Year_Factor_cols1, dtype: int64

In [9]:
test['site_eui']=np.where((test['site_eui_cols1'].notnull() ) & 
                          (test['Year_Factor_cols1']>3 ) & 
                          (test['Year_Factor_cols1'] <6) , 
                           test['site_eui_cols1_updated']*0.5 + test['site_eui_sub']*0.5  ,
                np.where( (test['site_eui_cols1'].notnull()) &
                          (test['Year_Factor_cols1'] >5) ,
                           test['site_eui_cols1_updated']*0.7+test['site_eui_sub']*0.3 ,
                           test['site_eui_sub'] ))

sub_pp = test[['id','site_eui']].copy()

# Prediction data with no previous value using some pseudo label

In [10]:
train = pd.read_csv('../input/widsdatathon2022/train.csv')
print(train.shape ,'before dedup')
cols4dup = [ c for c in train.columns if c not in ['Year_Factor', 'id'] ]
train = train[~train.duplicated(subset=cols4dup,keep='first')]
print(train.shape ,'after dedup')

test = pd.read_csv('../input/widsdatathon2022/test.csv')
test = test.merge(sub_pp,on='id',how='left')

cols1=['State_Factor','building_class','facility_type','floor_area','year_built']
train['source'] = 'train'
test['source']  = 'test'
df = pd.concat([train,test],0,ignore_index=True)
df=df.sort_values(by=cols1+['Year_Factor'])

# agregate by meteo station so same location 
min_temp = [col for col in df.columns if 'min_temp' in col]
max_temp = [col for col in df.columns if 'max_temp' in col]
avg_temp = [col for col in df.columns if 'avg_temp' in col]

df['min_temp1'] = df[min_temp].min(axis=1)
df['max_temp1'] = df[max_temp].min(axis=1)
df['avg_temp1'] = df[avg_temp].min(axis=1)

df['min_temp2'] = df[min_temp].max(axis=1)
df['max_temp2'] = df[max_temp].max(axis=1)
df['avg_temp2'] = df[avg_temp].max(axis=1)

df['min_temp3'] = df[min_temp].mean(axis=1)
df['max_temp3'] = df[max_temp].mean(axis=1)
df['avg_temp3'] = df[avg_temp].mean(axis=1)

df['min_temp4'] = df[min_temp].std(axis=1)
df['max_temp4'] = df[max_temp].std(axis=1)
df['avg_temp4'] = df[avg_temp].std(axis=1)

df['min_temp5'] = df[min_temp].skew(axis=1)
df['max_temp5'] = df[max_temp].skew(axis=1)
df['avg_temp5'] = df[avg_temp].skew(axis=1)


cols1=['State_Factor','building_class','facility_type','floor_area','year_built']

# lag from current values (just to identify row with no lag)
field_liste=['site_eui']
for i in [1] :
    for field in field_liste : 
        df[f'{field}_lag{i}'] = df.groupby(cols1)[field].shift(i)
        
#extra features
df['energy_star_rating_bin'] = pd.qcut(df['energy_star_rating'],5)

grp1 = df.groupby(min_temp,as_index=True).agg({
            "floor_area": ['mean', 'max','min','sum']  ,
            "year_built": ['mean', 'max','min'] ,
            "energy_star_rating" : ['min','max','mean', ] ,
            "id" : "count"
            })
grp1.columns = ["_".join(x) for x in grp1.columns.ravel()  ]
print(df.shape)
df = df.merge(grp1,on=min_temp,how='left')
print(df.shape)

df = df.sort_values(by='id')


(75757, 64) before dedup
(75718, 64) after dedup


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  del sys.path[0]
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:59: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.


(85423, 82)
(85423, 93)


In [11]:
df.head()

,Year_Factor,State_Factor,building_class,facility_type,floor_area,year_built,energy_star_rating,ELEVATION,january_min_temp,january_avg_temp,...,floor_area_max,floor_area_min,floor_area_sum,year_built_mean,year_built_max,year_built_min,energy_star_rating_min,energy_star_rating_max,energy_star_rating_mean,id_count
1260,1,State_1,Commercial,Grocery_store_or_food_market,61242.0,1942.0,11.0,2.4,36,50.5,...,1978104.0,41000.0,95093036.0,1943.304734,2008.0,0.0,6.0,100.0,76.44898,358
6106,1,State_1,Commercial,Warehouse_Distribution_or_Shipping_center,274000.0,1955.0,45.0,1.8,36,50.5,...,1978104.0,41000.0,95093036.0,1943.304734,2008.0,0.0,6.0,100.0,76.44898,358
5532,1,State_1,Commercial,Retail_Enclosed_mall,280025.0,1951.0,97.0,1.8,36,50.5,...,1978104.0,41000.0,95093036.0,1943.304734,2008.0,0.0,6.0,100.0,76.44898,358
449,1,State_1,Commercial,Education_Other_classroom,55325.0,1980.0,46.0,1.8,36,50.5,...,1978104.0,41000.0,95093036.0,1943.304734,2008.0,0.0,6.0,100.0,76.44898,358
6218,1,State_1,Commercial,Warehouse_Nonrefrigerated,66000.0,1985.0,100.0,2.4,36,50.5,...,1978104.0,41000.0,95093036.0,1943.304734,2008.0,0.0,6.0,100.0,76.44898,358


In [12]:
train1 = df.loc[(df['source']=='train')]
train2 = df.loc[(df['source']=='test') & ( df['site_eui_lag1'].notnull())]
train2 = train2.sample(frac=0.15,random_state=SEED)
train = pd.concat([train1,train2])
print(train.shape,train1.shape,train2.shape)
train.to_csv('train_h2O.csv',index=False)
test = df.loc[( df['site_eui_lag1'].isna()) & (df['source']=='test')]
print(test.shape)
test.to_csv('test_h2O.csv',index=False)


(77066, 93) (75718, 93) (1348, 93)
(715, 93)


In [13]:
h2o.init()
train =  h2o.import_file('train_h2O.csv')
test  =  h2o.import_file('test_h2O.csv')
x = train.columns
y = 'site_eui'
x.remove(y)
x.remove("id") 
x = [ c for c in x if 'Year_Factor__' not in c ]
x.remove('site_eui_lag1')
x.remove('source')
x

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.13" 2021-10-19; OpenJDK Runtime Environment (build 11.0.13+8-Ubuntu-0ubuntu1.20.04); OpenJDK 64-Bit Server VM (build 11.0.13+8-Ubuntu-0ubuntu1.20.04, mixed mode, sharing)
  Starting server from /opt/conda/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpyysnim3k
  JVM stdout: /tmp/tmpyysnim3k/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpyysnim3k/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.34.0.8
H2O_cluster_version_age:,1 month and 15 days
H2O_cluster_name:,H2O_from_python_unknownUser_u48v1m
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,4 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


['Year_Factor',
 'State_Factor',
 'building_class',
 'facility_type',
 'floor_area',
 'year_built',
 'energy_star_rating',
 'ELEVATION',
 'january_min_temp',
 'january_avg_temp',
 'january_max_temp',
 'february_min_temp',
 'february_avg_temp',
 'february_max_temp',
 'march_min_temp',
 'march_avg_temp',
 'march_max_temp',
 'april_min_temp',
 'april_avg_temp',
 'april_max_temp',
 'may_min_temp',
 'may_avg_temp',
 'may_max_temp',
 'june_min_temp',
 'june_avg_temp',
 'june_max_temp',
 'july_min_temp',
 'july_avg_temp',
 'july_max_temp',
 'august_min_temp',
 'august_avg_temp',
 'august_max_temp',
 'september_min_temp',
 'september_avg_temp',
 'september_max_temp',
 'october_min_temp',
 'october_avg_temp',
 'october_max_temp',
 'november_min_temp',
 'november_avg_temp',
 'november_max_temp',
 'december_min_temp',
 'december_avg_temp',
 'december_max_temp',
 'cooling_degree_days',
 'heating_degree_days',
 'precipitation_inches',
 'snowfall_inches',
 'snowdepth_inches',
 'avg_temp',
 'days_bel

In [14]:
aml = H2OAutoML(max_runtime_secs= 555, 
                max_models = 10,
                seed=1,
                stopping_metric='RMSE',
                include_algos = [ "GLM", "DRF","XGBoost","GBM","StackedEnsemble"])

aml.train(x=x, y=y, training_frame=train  ) 

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%
Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_AllModels_1_AutoML_1_20220301_145820

No model summary for this model

ModelMetricsRegressionGLM: stackedensemble
** Reported on train data. **

MSE: 1323.148688358683
RMSE: 36.375110836376614
MAE: 18.975314323922486
RMSLE: NaN
R^2: 0.6260319264784608
Mean Residual Deviance: 1323.148688358683
Null degrees of freedom: 9917
Residual degrees of freedom: 9911
Null deviance: 35091451.88706109
Residual deviance: 13122988.69114142
AIC: 99450.36309265549

ModelMetricsRegressionGLM: stackedensemble
** Reported on cross-validation data. **

MSE: 1720.5560287851313
RMSE: 41.47958568724055
MAE: 20.978218153774254
RMSLE: NaN
R^2: 0.49059182311822935
Mean Residual Deviance: 1720.5560287851313
Null degrees of freedom: 77065
Residual degrees of freedom: 77059
Null deviance: 260308871.05975273
Residual deviance: 13259637

In [15]:
# Get AutoML event log
log = aml.event_log

# Get training timing info
info = aml.training_info

In [16]:
lb = aml.leaderboard
lb

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
StackedEnsemble_AllModels_1_AutoML_1_20220301_145820,1720.56,41.4796,1720.56,20.9782,nan
StackedEnsemble_AllModels_2_AutoML_1_20220301_145820,1720.99,41.4848,1720.99,20.9822,nan
StackedEnsemble_BestOfFamily_3_AutoML_1_20220301_145820,1733.14,41.631,1733.14,20.9931,nan
StackedEnsemble_AllModels_3_AutoML_1_20220301_145820,1733.43,41.6345,1733.43,20.7912,0.443665
StackedEnsemble_BestOfFamily_2_AutoML_1_20220301_145820,1733.45,41.6347,1733.45,20.9945,nan
StackedEnsemble_BestOfFamily_1_AutoML_1_20220301_145820,1740.41,41.7182,1740.41,21.0264,nan
StackedEnsemble_BestOfFamily_4_AutoML_1_20220301_145820,1759.72,41.9491,1759.72,20.9498,0.441932
StackedEnsemble_BestOfFamily_5_AutoML_1_20220301_145820,1784.83,42.2472,1784.83,21.5697,0.47171
XGBoost_1_AutoML_1_20220301_145820,1787.49,42.2787,1787.49,21.667,nan
GBM_1_AutoML_1_20220301_145820,1815.35,42.6069,1815.35,21.1645,nan


In [17]:
preds = aml.predict(test)

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


In [18]:
## create submission
sample_submission=sub_pp.copy()
test = pd.read_csv('test_h2O.csv')
test['site_eui_pp'] = h2o.as_list(preds)['predict']
sample_submission=sample_submission.merge(test[['id','site_eui_pp']],on='id',how='left')
sample_submission['site_eui'] = np.where( sample_submission['site_eui_pp'].isna(),
                                          sample_submission['site_eui'],
                                          sample_submission['site_eui_pp']*0.75+sample_submission['site_eui']*0.25   )
sample_submission[['id','site_eui']].to_csv('sub_final_wids2022.csv',index=False)
print(sample_submission.shape)

(9705, 3)
